In [2]:
import pandas as pd

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 46.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=675392355da26fd1da063d8311bdd32e906aff87625fa677a9a82b2c90c55787
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Using Pandas read sqoop data set and flume dataset

In [3]:
df_sqp= pd.read_csv('/content/part-m-00000(2)', header = None)
df_sqp

0   1         2                              3
0    01/02/2022  10     DEBUG         Harmless debug Message
1    02/02/2022  30   WARNING                  Its a Warning
2    03/02/2022  20      INFO            Just an information
3    04/02/2022  40     ERROR  Did you try to divide by zero
4    05/02/2022  50  CRITICAL               Internet is down
..          ...  ..       ...                            ...
145  01/02/2022  10     DEBUG         Harmless debug Message
146  02/02/2022  30   WARNING                  Its a Warning
147  03/02/2022  20      INFO            Just an information
148  04/02/2022  40     ERROR  Did you try to divide by zero
149  05/02/2022  50  CRITICAL               Internet is down

[150 rows x 4 columns]

In [4]:
df_flume = pd.read_csv('/content/FlumeData.1644137030799', header = None)
df_flume

0   1         2                              3
0  06/02/2022  20      INFO            Just an information
1  06/02/2022  40     ERROR  Did you try to divide by zero
2  06/02/2022  20      INFO            Just an information
3  06/02/2022  40     ERROR  Did you try to divide by zero
4  06/02/2022  50  CRITICAL               Internet is down
5  06/02/2022  30   WARNING                  Its a Warning
6  06/02/2022  30   WARNING                  Its a Warning
7  06/02/2022  50  CRITICAL               Internet is down
8  06/02/2022  30   WARNING                  Its a Warning
9  06/02/2022  10     DEBUG         Harmless debug Message

## Merge the DataFrame flume and sqoop using pandas 

In [5]:
df=pd.concat([df_sqp, df_flume], axis=0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 0 to 9
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       160 non-null    object
 1   1       160 non-null    int64 
 2   2       160 non-null    object
 3   3       160 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.2+ KB


# Data type of columns


In [7]:
df.dtypes

0    object
1     int64
2    object
3    object
dtype: object

#Total columns in merged data set

In [9]:
df.columns

Int64Index([0, 1, 2, 3], dtype='int64')

# Give the column name in dataset

In [10]:
df.columns = ['date', 'duration', 'category', 'activity']

In [11]:
df.head()

date  duration  category                       activity
0  01/02/2022        10     DEBUG         Harmless debug Message
1  02/02/2022        30   WARNING                  Its a Warning
2  03/02/2022        20      INFO            Just an information
3  04/02/2022        40     ERROR  Did you try to divide by zero
4  05/02/2022        50  CRITICAL               Internet is down

# OneHot encoding of activity column

In [16]:
df = pd.get_dummies(df,columns=['activity'],drop_first=True)

In [17]:
df.head()

date  duration  ...  activity_Its a Warning  activity_Just an information
0  01/02/2022        10  ...                       0                             0
1  02/02/2022        30  ...                       1                             0
2  03/02/2022        20  ...                       0                             1
3  04/02/2022        40  ...                       0                             0
4  05/02/2022        50  ...                       0                             0

[5 rows x 7 columns]

# Label encoding of category column

In [12]:
df['category'].unique()

array(['DEBUG', 'WARNING', 'INFO', 'ERROR', 'CRITICAL'], dtype=object)

In [13]:
label_map = {'category' : {'DEBUG' : 0, 'WARNING':1, 'INFO':2, 'ERROR':3, 'CRITICAL':4}}

In [14]:
df.replace(label_map, inplace=True)

In [15]:
df.head()

date  duration  category                       activity
0  01/02/2022        10         0         Harmless debug Message
1  02/02/2022        30         1                  Its a Warning
2  03/02/2022        20         2            Just an information
3  04/02/2022        40         3  Did you try to divide by zero
4  05/02/2022        50         4               Internet is down

# Now bulid a spark session 

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Classification').getOrCreate()

# Apply Spark on data set

In [19]:
df_spk=spark.createDataFrame(df)

In [20]:
df_spk.show()

+----------+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+
|      date|duration|category|activity_Harmless debug Message|activity_Internet is down|activity_Its a Warning|activity_Just an information|
+----------+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+
|01/02/2022|      10|       0|                              1|                        0|                     0|                           0|
|02/02/2022|      30|       1|                              0|                        0|                     1|                           0|
|03/02/2022|      20|       2|                              0|                        0|                     0|                           1|
|04/02/2022|      40|       3|                              0|                        0|                     0|                           0|
|05/02/2022| 

In [21]:
df_spk.printSchema()

root
 |-- date: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- category: long (nullable = true)
 |-- activity_Harmless debug Message: long (nullable = true)
 |-- activity_Internet is down: long (nullable = true)
 |-- activity_Its a Warning: long (nullable = true)
 |-- activity_Just an information: long (nullable = true)



In [23]:
df_spk=df_spk.drop('date')

In [24]:
df_spk.show()

+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+
|duration|category|activity_Harmless debug Message|activity_Internet is down|activity_Its a Warning|activity_Just an information|
+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+
|      10|       0|                              1|                        0|                     0|                           0|
|      30|       1|                              0|                        0|                     1|                           0|
|      20|       2|                              0|                        0|                     0|                           1|
|      40|       3|                              0|                        0|                     0|                           0|
|      50|       4|                              0|                        1|             

In [25]:
df_spk.printSchema()

root
 |-- duration: long (nullable = true)
 |-- category: long (nullable = true)
 |-- activity_Harmless debug Message: long (nullable = true)
 |-- activity_Internet is down: long (nullable = true)
 |-- activity_Its a Warning: long (nullable = true)
 |-- activity_Just an information: long (nullable = true)



# Using VectorAssembler to add feature column where all input columns combine as list 


# feature columns mai target column use nhi krte hai

In [26]:
#use VectorAssembler to combine the training features
from pyspark.ml.feature import VectorAssembler
df_spk.columns

['duration',
 'category',
 'activity_Harmless debug Message',
 'activity_Internet is down',
 'activity_Its a Warning',
 'activity_Just an information']

In [27]:
assembler = VectorAssembler(
    inputCols = ['duration', 'activity_Harmless debug Message', 'activity_Internet is down', 'activity_Its a Warning', 'activity_Just an information'],
    outputCol = 'features'
)

In [28]:
df_spk = assembler.transform(df_spk)

In [29]:
df_spk.show()

+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|duration|category|activity_Harmless debug Message|activity_Internet is down|activity_Its a Warning|activity_Just an information|            features|
+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      30|       1|                              0|                        0|                     1|                           0|(5,[0,3],[30.0,1.0])|
|      20|       2|                              0|                        0|                     0|                           1|(5,[0,4],[20.0,1.0])|
|      40|       3|                              0|                        0|                 

#Split the train and test in pyspark 

In [30]:
#splitting the data 
trainingData, testData = df_spk.randomSplit([0.7,0.3])

CHECKING TRAIN DATA

In [31]:
trainingData.show()

+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|duration|category|activity_Harmless debug Message|activity_Internet is down|activity_Its a Warning|activity_Just an information|            features|
+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                 

CHECKING TEST DATA

In [32]:
testData.show()

+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|duration|category|activity_Harmless debug Message|activity_Internet is down|activity_Its a Warning|activity_Just an information|            features|
+--------+--------+-------------------------------+-------------------------+----------------------+----------------------------+--------------------+
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                     0|                           0|(5,[0,1],[10.0,1.0])|
|      10|       0|                              1|                        0|                 

# Apply DecisionTreeClassifier into dataset

In [33]:
from pyspark.ml import pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Built Model using DecsionTree
###Create model labelCol = Target col, featureCol= features

In [35]:
#create model
model = DecisionTreeClassifier(labelCol='category',featuresCol='features')

In [36]:
modelfit = model.fit(trainingData)

In [37]:
predictions = modelfit.transform(testData)

In [38]:
predictions.columns

['duration',
 'category',
 'activity_Harmless debug Message',
 'activity_Internet is down',
 'activity_Its a Warning',
 'activity_Just an information',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [40]:
predictions.select("prediction",'category').show()

+----------+--------+
|prediction|category|
+----------+--------+
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       2.0|       2|
|       2.0|       2|
|       1.0|       1|
|       1.0|       1|
|       3.0|       3|
|       3.0|       3|
|       3.0|       3|
|       3.0|       3|
|       3.0|       3|
|       3.0|       3|
|       3.0|       3|
+----------+--------+
only showing top 20 rows



#### Model evalution

In [41]:
#evaluate
eval = MulticlassClassificationEvaluator(
    labelCol='category',
    predictionCol='prediction',
    metricName="accuracy"
)

In [42]:
acc = eval.evaluate(predictions)

In [43]:
print(acc)

1.0


WE GET 100 PERCENT ACCURACY HERE BECAUSE WE HAVE LESS DATA TO WORK WITH 